# 特殊な加工・可視化を行なう10本 Knock

## 大容量CSV data を扱ってみよう
容量無制限に処理できるものではない為、Memory容量との兼ね合いなどを考慮しつつ工夫する必要がでてくる、

※時系列Data を大容量Data と仮定して使用し演習を実施。Data が小さくても Code の書き方と結果を確認し Image を掴む。

In [2]:
# File を読み込む
import pandas as pd
df = pd.read_csv('data/chapter8/person_count_out_0001_2021011509.csv')
df

,id,place,receive_time,sensor_num,in1,out1,state1,in2,out2,state2
0,0,1,2021-01-15 09:00:00.144,2,508,73,0,73,508,0
1,1,1,2021-01-15 09:00:01.146,2,508,73,0,73,508,0
2,2,1,2021-01-15 09:00:02.161,2,508,73,0,73,508,0
3,3,1,2021-01-15 09:00:03.176,2,508,73,0,73,508,0
4,4,1,2021-01-15 09:00:04.192,2,508,73,0,73,508,0
...,...,...,...,...,...,...,...,...,...,...
3535,3535,1,2021-01-15 09:59:55.054,2,782,156,0,156,782,0
3536,3536,1,2021-01-15 09:59:56.07,2,782,156,0,156,782,0
3537,3537,1,2021-01-15 09:59:57.085,2,782,156,0,156,782,0
3538,3538,1,2021-01-15 09:59:58.101,2,782,156,0,156,782,0


仮に Data数が 100万~1000万以上の Data になると pandas.read_csv() で一括で読み込もうとすると OOM Error で Program が落ちると想定される。
※ OOM Error: Memory不足による Error

In [3]:
# OOM Error を回避する為に引数で桁数を指定して読み込む
for df in pd.read_csv('data/chapter8/person_count_out_0001_2021011509.csv', chunksize=512):
    print(df.shape)

(512, 10)
(512, 10)
(512, 10)
(512, 10)
(512, 10)
(512, 10)
(468, 10)


`chunksize=` 明示的に指定すると指定した行数ごとに CSV File を読み込む

In [5]:
# 読み込んだ Dataに対して何らかの処理を行なった上で、別File に保存してみる
i = 0
for df in pd.read_csv('data/chapter8/person_count_out_0001_2021011509.csv', chunksize=64):  # Chunksize を指定して読み込み
    df['processed_per_chunk'] = True  # column を追加
    df.to_csv('data/chapter8/processed_big_data.csv', mode='a', index_label=False, header=i == 0)  # 新しい File に保存していく
    # media='a' と指定することで同じ File に対して追記して保存する。（※'a': append の頭文字）
    # header=i == 0 とすることで i == 0 の時だけ header ありで出力するようにしている

    i += 1

In [7]:
# 結果の確認
from IPython.display import display
df = pd.read_csv('data/chapter8/processed_big_data.csv')
display(df)

,id,place,receive_time,sensor_num,in1,out1,state1,in2,out2,state2,processed_per_chunk
0,0,1,2021-01-15 09:00:00.144,2,508,73,0,73,508,0,True
1,1,1,2021-01-15 09:00:01.146,2,508,73,0,73,508,0,True
2,2,1,2021-01-15 09:00:02.161,2,508,73,0,73,508,0,True
3,3,1,2021-01-15 09:00:03.176,2,508,73,0,73,508,0,True
4,4,1,2021-01-15 09:00:04.192,2,508,73,0,73,508,0,True
...,...,...,...,...,...,...,...,...,...,...,...
3535,3535,1,2021-01-15 09:59:55.054,2,782,156,0,156,782,0,True
3536,3536,1,2021-01-15 09:59:56.07,2,782,156,0,156,782,0,True
3537,3537,1,2021-01-15 09:59:57.085,2,782,156,0,156,782,0,True
3538,3538,1,2021-01-15 09:59:58.101,2,782,156,0,156,782,0,True
